In [ ]:
# Instala la herramienta Consola del archivo ISIC
!pip install isic-cli #Sacada desde el repositorio de Github https://github.com/ImageMarkup/isic-cli/blob/master/README.md

# Se actualizan las librerías a versiones compatibles para resolver los conflictos.
!pip install tensorflow==2.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colabsqlviz 0.2.5 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.
grpcio-status 1.75.1 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.


In [ ]:
#Descargar Imagenes + Metadatos
!isic image download --search "" --collections "249" --limit 0 BCN20000/ # Imágenes y Metadatos de BCN20000

If you have been granted special permissions, logging in with `isic user login` might return more data.


Successfully downloaded 18,946 images to BCN20000/.
Successfully wrote 18,946 metadata records to BCN20000/metadata.csv.
Successfully wrote attributions to BCN20000/attribution.txt.
Successfully wrote 1 license(s) to BCN20000/licenses.


In [ ]:
#Visualización de Imagenes Descargadas
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(15,15))
carpeta = '/content/BCN20000/' #Visualización de Imágenes para Entrenamiento
imagenes = os.listdir(carpeta)

for i , imagen in enumerate(imagenes[: 25]):
  plt.subplot(5, 5, i + 1)
  imagen = mpimg.imread(carpeta + '/' + imagen)
  plt.imshow(imagen)
  plt.axis("off")
plt.show()

In [ ]:
import csv
def cargar_metadatos(archivo):
    datos = {
        'isic_id': [],
        'diagnosis_1': []  # Las etiquetas serán 0 o 1
    }
    etiquetas_validas = {'Benign': 0, 'Malignant': 1}  # Mapeamos las etiquetas a 0 y 1

    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Evita las columnas cabeceras.

            diagnosis_idx =7  # Cambiar al índice correspondiente si es necesario
            #7 para BCN20000
            for fila in reader:
                if len(fila) > diagnosis_idx:
                    diagnosis = fila[diagnosis_idx]
                    if diagnosis in etiquetas_validas:  # Solo almacenar Benign y Malignant
                        datos['isic_id'].append(fila[0])  # Almacena el ID de la imagen
                        datos['diagnosis_1'].append(etiquetas_validas[diagnosis])  # Asignamos la etiqueta 0 o 1
        print(f"Datos extraídos y filtrados correctamente. Se encontraron {len(datos['isic_id'])} registros válidos.")
        return datos

    except FileNotFoundError:
        print(f"Error: Archivo {archivo} no encontrado")
        return None

    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def organizar_imagenes_en_carpetas(carpeta_origen, metadatos, carpeta_destino_base):
    # Crea las carpetas de destino
    for split in ['train', 'test']:
        for class_name in ['benign', 'malignant']:
            os.makedirs(os.path.join(carpeta_destino_base, split, class_name), exist_ok=True)

    # Separa los IDs por clase
    ids_benignos = []
    ids_malignos = []
    for id_img, etiqueta in zip(metadatos['isic_id'], metadatos['diagnosis_1']):
        if etiqueta == 0:  # Benign
            ids_benignos.append(id_img)
        elif etiqueta == 1: # Malignant
            ids_malignos.append(id_img)

    # Divide los IDs en conjuntos de entrenamiento y prueba para cada clase
    ids_benignos_train, ids_benignos_test = train_test_split(ids_benignos, test_size=0.3, random_state=42)
    ids_malignos_train, ids_malignos_test = train_test_split(ids_malignos, test_size=0.3, random_state=42)

    # Función auxiliar para copiar archivos
    def copiar_archivos(ids, carpeta_destino):
        for id_img in tqdm(ids, desc=f'Copiando a {os.path.basename(carpeta_destino)}'):
            nombre_archivo = f"{id_img}.jpg"
            ruta_origen = os.path.join(carpeta_origen, nombre_archivo)
            ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
            if os.path.exists(ruta_origen):
                shutil.copy(ruta_origen, ruta_destino)
            else:
                print(f"Advertencia: No se encontró el archivo {ruta_origen}")

    #  Copia los archivos a sus directorios finales
    print("Copiando imágenes de entrenamiento benignas...")
    copiar_archivos(ids_benignos_train, os.path.join(carpeta_destino_base, 'train', 'benign'))
    print("Copiando imágenes de entrenamiento malignas...")
    copiar_archivos(ids_malignos_train, os.path.join(carpeta_destino_base, 'train', 'malignant'))
    print("Copiando imágenes de prueba benignas...")
    copiar_archivos(ids_benignos_test, os.path.join(carpeta_destino_base, 'test', 'benign'))
    print("Copiando imágenes de prueba malignas...")
    copiar_archivos(ids_malignos_test, os.path.join(carpeta_destino_base, 'test', 'malignant'))
    print("\nImágenes organizadas correctamente en carpetas 'train' y 'test' sin fugas de datos.")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# Lineamiento: L2 muy reducido
L2_REG = 1e-5

def Arq1_Refactorizada(input_shape=(224, 224, 3), num_classes=2):

    # Validar restricción SCCE
    if num_classes < 2:
        raise ValueError("num_classes debe ser 2 o más para sparse_categorical_crossentropy")

    model = models.Sequential([
        layers.Input(shape=input_shape),

        # Bloque 1
        layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False), # Bias innecesario antes de BN
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque 2
        layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(), # Estabilizar la salida de GAP
        layers.Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(L2_REG)),
        layers.Dropout(0.5), # Mantener el Dropout fuerte

        # Capa de salida
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

def Arq1_Optimizada_Refactorizada(input_shape=(224, 224, 3), num_classes=2):

    if num_classes < 2:
        raise ValueError("num_classes debe ser 2 o más para sparse_categorical_crossentropy")

    model = models.Sequential([
        layers.Input(shape=input_shape),

        # Bloque 1
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False), # Bias innecesario antes de BN
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque 2
        layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación
        layers.GlobalAveragePooling2D(),

        layers.Dense(num_classes, activation='softmax')
    ])

    return model

def Arq1_CapaPlana_Refactorizada(input_shape=(224, 224, 3), num_classes=2):

    if num_classes < 2:
        raise ValueError("num_classes debe ser 2 o más para sparse_categorical_crossentropy")

    model = models.Sequential([
        layers.Input(shape=input_shape),

        # Bloque 1
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False), # Bias innecesario antes de BN
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque 2
        layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación

        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(L2_REG)),
        layers.Dropout(0.5), # Se preserva el Dropout fuerte del cabezal
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

def Arq2_Refactorizada(input_shape=(224, 224, 3), num_classes=2):
    if num_classes < 2:
        raise ValueError("num_classes debe ser 2 o más para sparse_categorical_crossentropy")

    model = models.Sequential([
        layers.Input(shape=input_shape),

        # Bloque 1
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False), # Bias innecesario antes de BN
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque 2
        layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación
        layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(L2_REG)),
        layers.Dropout(0.5), # Se preserva el Dropout fuerte del cabezal

        # Capa de salida
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

def Arq3_Refactorizada(input_shape=(224, 224, 3), num_classes=2):

    width_factor = 1.1

    if num_classes < 2:
        raise ValueError("num_classes debe ser 2 o más para sparse_categorical_crossentropy")

    model = models.Sequential([
        layers.Input(shape=input_shape),

        # Bloque 1
        layers.Conv2D(int(32 * width_factor), (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),


        # Bloque 2
        layers.Conv2D(int(64 * width_factor), (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(int(64 * width_factor), (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque 3
        layers.Conv2D(int(128 * width_factor), (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(int(128 * width_factor), (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(int(128 * width_factor), (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG), use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación
        layers.GlobalAveragePooling2D(),

        layers.BatchNormalization(),
        layers.Dense(int(512 * width_factor), activation='relu',
                     kernel_regularizer=regularizers.l2(L2_REG)),
        layers.Dropout(0.5),

        # Capa de salida
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

L2_REG_SEP = 1e-5

def Arq4_Refactorizada(input_shape=(224, 224, 3), num_classes=2):
    if num_classes < 2:
        raise ValueError("num_classes debe ser 2 o más para sparse_categorical_crossentropy")

    model = models.Sequential([
        # Bloque de Entrada
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(L2_REG),
                      activation=None,
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),

        # Bloque Separable 1
        layers.SeparableConv2D(64, (3, 3), padding='same',
                               depthwise_initializer='he_normal',
                               pointwise_initializer='he_normal',
                               depthwise_regularizer=regularizers.l2(L2_REG_SEP),
                               pointwise_regularizer=regularizers.l2(L2_REG_SEP),
                               activation=None,
                               use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque Separable 2
        layers.SeparableConv2D(128, (3, 3), padding='same',
                               depthwise_initializer='he_normal',
                               pointwise_initializer='he_normal',
                               depthwise_regularizer=regularizers.l2(L2_REG_SEP),
                               pointwise_regularizer=regularizers.l2(L2_REG_SEP),
                               activation=None,
                               use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Bloque Separable 3
        layers.SeparableConv2D(256, (3, 3), padding='same',
                               depthwise_initializer='he_normal',
                               pointwise_initializer='he_normal',
                               depthwise_regularizer=regularizers.l2(L2_REG_SEP),
                               pointwise_regularizer=regularizers.l2(L2_REG_SEP),
                               activation=None,
                               use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),

        # Cabezal de Clasificación
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Creación de los modelos
Modelo_1_Normal_Refac = Arq1_Refactorizada()

Modelo_1_Optimizado_Refac = Arq1_Optimizada_Refactorizada()

Modelo_1_CapaPlana_Refac = Arq1_CapaPlana_Refactorizada()

Modelo_2_Refac = Arq2_Refactorizada()

Modelo_3_Refac = Arq3_Refactorizada()

Modelo_4_Refac = Arq4_Refactorizada()

---
APARTADO PARA GUARDAR Y DESCARGAR LOS MODELOS ENTRENADOS

---
APARTADO PARA CONVERTIR REGISTROS CSV A EXCEL

In [ ]:
import os
import shutil

# Eliminar el directorio del dataset antiguo para evitar conflictos y asegurar una regeneración limpia
if os.path.exists('/content/dataset'):
    shutil.rmtree('/content/dataset')
    print("Directorio '/content/dataset' anterior eliminado.")
else:
    print("El directorio '/content/dataset' no existía, no fue necesaria ninguna limpieza.")

El directorio '/content/dataset' no existía, no fue necesaria ninguna limpieza.


In [ ]:
# Asumiendo que los metadatos ya han sido cargados correctamente
metadatos = cargar_metadatos("/content/BCN20000/metadata.csv")

Datos extraídos y filtrados correctamente. Se encontraron 16702 registros válidos.


In [ ]:
organizar_imagenes_en_carpetas('/content/BCN20000/', metadatos, '/content/dataset')

Copiando imágenes de entrenamiento benignas...


Copiando a benign: 100%|██████████| 5481/5481 [00:00<00:00, 8896.28it/s]


Copiando imágenes de entrenamiento malignas...


Copiando a malignant: 100%|██████████| 6209/6209 [00:00<00:00, 8698.84it/s]


Copiando imágenes de prueba benignas...


Copiando a benign: 100%|██████████| 2350/2350 [00:00<00:00, 8560.61it/s]


Copiando imágenes de prueba malignas...


Copiando a malignant: 100%|██████████| 2662/2662 [00:00<00:00, 8772.39it/s]


Imágenes organizadas correctamente en carpetas 'train' y 'test' sin fugas de datos.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def crear_generadores(carpeta_destino_base, batch_size=32):
  train_datagen = ImageDataGenerator(
      rescale=1./255,
      # Rangos Aumentados
      rotation_range=30, # Rotación.
      width_shift_range=0.2, # Traslación.
      height_shift_range=0.2,
      shear_range=20,
      zoom_range=[0.8, 1.2], # +-20% de zoom.
      vertical_flip=True, #Duplica la variabilidad.
      brightness_range=[0.8, 1.2], # Simula diferentes condiciones de iluminación.
      horizontal_flip=True,
      fill_mode='nearest',
      )
  # Generador de datos para validación
  test_datagen = ImageDataGenerator(rescale=1./255)

  # Crear generador para entrenamiento
  train_generator = train_datagen.flow_from_directory(
      os.path.join(carpeta_destino_base, 'train'),
      target_size=(224, 224),
      batch_size=batch_size, #32
      class_mode='sparse', # sparse_categorical_crossentropy
      shuffle=True
      )

  # Crear generador para validación
  validation_generator = test_datagen.flow_from_directory(
      os.path.join(carpeta_destino_base, 'test'),
      target_size=(224, 224),
      batch_size=batch_size,
      class_mode='sparse', # sparse_categorical_crossentropy
      shuffle=False # Correcto para validación
      )

  print("Generadores de datos (con aumentación agresiva) creados.")
  return train_generator, validation_generator

train_generator, validation_generator = crear_generadores('/content/dataset')

Found 11690 images belonging to 2 classes.
Found 5012 images belonging to 2 classes.
Generadores de datos (con aumentación agresiva) creados.


In [ ]:
import tensorflow as tf

# Optimizador
rmsopti = tf.keras.optimizers.RMSprop(
    learning_rate = 0.0005,
    rho = 0.9,
    momentum = 0.0,
    epsilon = 1e-07,
)

METRICS = ['accuracy']

# Pérdida
loss_Focal = 'sparse_categorical_crossentropy'


Modelo_1_Normal_Refac.compile(optimizer=rmsopti,
                              loss=loss_Focal,
                              metrics=METRICS)

Modelo_1_Optimizado_Refac.compile(optimizer=rmsopti,
                                  loss=loss_Focal,
                                  metrics=METRICS)

Modelo_1_CapaPlana_Refac.compile(optimizer=rmsopti,
                                 loss=loss_Focal,
                                 metrics=METRICS)

Modelo_2_Refac.compile(optimizer=rmsopti,
                       loss=loss_Focal,
                       metrics=METRICS)

Modelo_3_Refac.compile(optimizer=rmsopti,
                       loss=loss_Focal,
                       metrics=METRICS)

Modelo_4_Refac.compile(optimizer=rmsopti,
                       loss=loss_Focal,
                       metrics=METRICS)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard, CSVLogger, ReduceLROnPlateau

# Create CSVLogger callbacks for each model to save epoch data to a CSV file
csv_logger_1_normal = CSVLogger('registro_entrenamiento_modelo_1_normal.csv', separator=',', append=False)
csv_logger_1_opt = CSVLogger('registro_entrenamiento_modelo_1_opt.csv', separator=',', append=False)
csv_logger_1_cplana = CSVLogger('registro_entrenamiento_modelo_1_cplana.csv', separator=',', append=False)
csv_logger_2 = CSVLogger('registro_entrenamiento_modelo_2.csv', separator=',', append=False)
csv_logger_3 = CSVLogger('registro_entrenamiento_modelo_3.csv', separator=',', append=False)
csv_logger_4 = CSVLogger('registro_entrenamiento_modelo_4.csv', separator=',', append=False)


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', factor=0.5, patience=5, verbose=1)

Model1 = TensorBoard(log_dir='logs/Model1')
history = Modelo_1_Normal_Refac.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[csv_logger_1_normal, reduce_lr]
)

Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 0s 525ms/step - accuracy: 0.5933 - loss: 0.7380

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


366/366 ━━━━━━━━━━━━━━━━━━━━ 233s 602ms/step - accuracy: 0.5934 - loss: 0.7379 - val_accuracy: 0.6508 - val_loss: 0.6437 - learning_rate: 5.0000e-04
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 208s 569ms/step - accuracy: 0.6378 - loss: 0.6537 - val_accuracy: 0.6365 - val_loss: 0.6409 - learning_rate: 5.0000e-04
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 210s 574ms/step - accuracy: 0.6576 - loss: 0.6361 - val_accuracy: 0.6828 - val_loss: 0.6069 - learning_rate: 5.0000e-04
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 205s 559ms/step - accuracy: 0.6615 - loss: 0.6332 - val_accuracy: 0.6854 - val_loss: 0.6083 - learning_rate: 5.0000e-04
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 205s 561ms/step - accuracy: 0.6607 - loss: 0.6289 - val_accuracy: 0.6901 - val_loss: 0.5996 - learning_rate: 5.0000e-04
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 206s 562ms/step - accuracy: 0.6782 - loss: 0.6175 - val_accuracy: 0.6838 - val_loss: 0.6026 - learning_rate: 5.0000e-04
Epoch 7/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 207s 

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', factor=0.5, patience=5, verbose=1)

Modelo = TensorBoard(log_dir='logs/Model1o')
history = Modelo_1_Optimizado_Refac.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[csv_logger_1_opt, reduce_lr]
)

Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 229s 595ms/step - accuracy: 0.6288 - loss: 0.6621 - val_accuracy: 0.6311 - val_loss: 0.6516 - learning_rate: 5.0000e-04
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 210s 575ms/step - accuracy: 0.6458 - loss: 0.6398 - val_accuracy: 0.6540 - val_loss: 0.6213 - learning_rate: 5.0000e-04
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 212s 580ms/step - accuracy: 0.6475 - loss: 0.6332 - val_accuracy: 0.6720 - val_loss: 0.6277 - learning_rate: 5.0000e-04
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 209s 572ms/step - accuracy: 0.6529 - loss: 0.6277 - val_accuracy: 0.6299 - val_loss: 0.6534 - learning_rate: 5.0000e-04
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 214s 583ms/step - accuracy: 0.6605 - loss: 0.6237 - val_accuracy: 0.6566 - val_loss: 0.6238 - learning_rate: 5.0000e-04
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 212s 580ms/step - accuracy: 0.6522 - loss: 0.6295 - val_accuracy: 0.6598 - val_loss: 0.6257 - learning_rate: 5.0000e-04
Epoch 7/100
366/366 ━━━━━━━━━━━━━━

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', factor=0.5, patience=5, verbose=1)

Model1cplana = TensorBoard(log_dir='logs/Model1cplana')
history = Modelo_1_CapaPlana_Refac.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[csv_logger_1_cplana, reduce_lr]
)

Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 236s 611ms/step - accuracy: 0.5979 - loss: 0.7454 - val_accuracy: 0.6628 - val_loss: 0.6312 - learning_rate: 5.0000e-04
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 212s 578ms/step - accuracy: 0.6403 - loss: 0.6539 - val_accuracy: 0.6658 - val_loss: 0.6223 - learning_rate: 5.0000e-04
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 211s 577ms/step - accuracy: 0.6454 - loss: 0.6438 - val_accuracy: 0.6792 - val_loss: 0.6147 - learning_rate: 5.0000e-04
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 210s 575ms/step - accuracy: 0.6492 - loss: 0.6348 - val_accuracy: 0.6636 - val_loss: 0.6123 - learning_rate: 5.0000e-04
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 214s 586ms/step - accuracy: 0.6571 - loss: 0.6271 - val_accuracy: 0.5960 - val_loss: 0.6901 - learning_rate: 5.0000e-04
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 213s 582ms/step - accuracy: 0.6677 - loss: 0.6198 - val_accuracy: 0.6696 - val_loss: 0.6146 - learning_rate: 5.0000e-04
Epoch 7/100
366/366 ━━━━━━━━━━━━━━

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', factor=0.5, patience=5, verbose=1)

Model2 = TensorBoard(log_dir='logs/Model2')
history = Modelo_2_Refac.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[csv_logger_2, reduce_lr]
    )

Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 230s 604ms/step - accuracy: 0.5870 - loss: 0.7862 - val_accuracy: 0.6397 - val_loss: 0.6405 - learning_rate: 5.0000e-04
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 213s 581ms/step - accuracy: 0.6380 - loss: 0.6596 - val_accuracy: 0.6576 - val_loss: 0.6573 - learning_rate: 5.0000e-04
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 217s 591ms/step - accuracy: 0.6534 - loss: 0.6424 - val_accuracy: 0.6770 - val_loss: 0.6155 - learning_rate: 5.0000e-04
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 211s 575ms/step - accuracy: 0.6631 - loss: 0.6381 - val_accuracy: 0.6596 - val_loss: 0.6222 - learning_rate: 5.0000e-04
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 213s 583ms/step - accuracy: 0.6681 - loss: 0.6292 - val_accuracy: 0.6931 - val_loss: 0.6131 - learning_rate: 5.0000e-04
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 212s 579ms/step - accuracy: 0.6712 - loss: 0.6214 - val_accuracy: 0.6614 - val_loss: 0.6164 - learning_rate: 5.0000e-04
Epoch 7/100
366/366 ━━━━━━━━━━━━━━

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', factor=0.5, patience=5, verbose=1)

Model3 = TensorBoard(log_dir='logs/Model3')
history = Modelo_3_Refac.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[csv_logger_3, reduce_lr]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 254s 647ms/step - accuracy: 0.5992 - loss: 0.7696 - val_accuracy: 0.6844 - val_loss: 0.6287 - learning_rate: 5.0000e-04
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 213s 581ms/step - accuracy: 0.6433 - loss: 0.6759 - val_accuracy: 0.6472 - val_loss: 0.6956 - learning_rate: 5.0000e-04
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 212s 580ms/step - accuracy: 0.6470 - loss: 0.6539 - val_accuracy: 0.6580 - val_loss: 0.6351 - learning_rate: 5.0000e-04
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 213s 582ms/step - accuracy: 0.6517 - loss: 0.6438 - val_accuracy: 0.6870 - val_loss: 0.6232 - learning_rate: 5.0000e-04
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 213s 581ms/step - accuracy: 0.6726 - loss: 0.6327 - val_accuracy: 0.6792 - val_loss: 0.6256 - learning_rate: 5.0000e-04
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 211s 577ms/step - accuracy: 0.6735 - loss: 0.6301 - val_accuracy: 0.6977 - val_loss: 0.6135 - learning_rate: 5.0000e-04
Epoch 7/100
366/366 ━━━━━━━━━━━━━━

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', mode='max', factor=0.5, patience=5, verbose=1)

Model4 = TensorBoard(log_dir='logs/Model4')
history = Modelo_4_Refac.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    callbacks=[csv_logger_4, reduce_lr]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 248s 630ms/step - accuracy: 0.6358 - loss: 0.6562 - val_accuracy: 0.5950 - val_loss: 0.6865 - learning_rate: 5.0000e-04
Epoch 2/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 209s 571ms/step - accuracy: 0.6513 - loss: 0.6339 - val_accuracy: 0.6662 - val_loss: 0.6283 - learning_rate: 5.0000e-04
Epoch 3/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 211s 576ms/step - accuracy: 0.6522 - loss: 0.6354 - val_accuracy: 0.6842 - val_loss: 0.6263 - learning_rate: 5.0000e-04
Epoch 4/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 209s 572ms/step - accuracy: 0.6686 - loss: 0.6216 - val_accuracy: 0.6664 - val_loss: 0.6189 - learning_rate: 5.0000e-04
Epoch 5/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 215s 587ms/step - accuracy: 0.6569 - loss: 0.6278 - val_accuracy: 0.6911 - val_loss: 0.5994 - learning_rate: 5.0000e-04
Epoch 6/100
366/366 ━━━━━━━━━━━━━━━━━━━━ 212s 578ms/step - accuracy: 0.6739 - loss: 0.6167 - val_accuracy: 0.6728 - val_loss: 0.6263 - learning_rate: 5.0000e-04
Epoch 7/100
366/366 ━━━━━━━━━━━━━━

In [ ]:
import os

# Crear la carpeta para guardar los modelos si no existe
os.makedirs('/content/MisModelos', exist_ok=True)

# Guardar los modelos entrenados
print("Guardando modelos...")
Modelo_1_Normal_Refac.save('/content/MisModelos/Modelo_1_Normal.h5')
Modelo_1_Optimizado_Refac.save('/content/MisModelos/Modelo_1_Optimizado.h5')
Modelo_1_CapaPlana_Refac.save('/content/MisModelos/Modelo_1_CapaPlana.h5')
Modelo_2_Refac.save('/content/MisModelos/Modelo_2.h5')
Modelo_3_Refac.save('/content/MisModelos/Modelo_3.h5')
Modelo_4_Refac.save('/content/MisModelos/Modelo_4.h5')

print("Modelos guardados exitosamente en /content/MisModelos/")

Guardando modelos...
Modelos guardados exitosamente en /content/MisModelos/


In [ ]:
from google.colab import files
import os

# Ruta del modelo específico que se guardó en la celda anterior
modelo_path = '/content/MisModelos/Modelo_1_Normal.h5'

print(f"Intentando descargar el modelo: {os.path.basename(modelo_path)}")

# Verificar si el archivo del modelo existe antes de iniciar la descarga
if os.path.exists(modelo_path):
    files.download(modelo_path)
    print("\nDescarga completada exitosamente.")
else:
    print(f"\nError: No se encontró el archivo del modelo en la ruta especificada: {modelo_path}")
    print("Por favor, asegúrate de que la celda anterior para guardar el modelo se haya ejecutado sin errores.")

Intentando descargar el modelo: Modelo_4.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Descarga completada exitosamente.


In [ ]:
import pandas as pd
import os

def convertir_csv_a_excel(ruta_csv, ruta_excel):
    """
    Lee un archivo CSV de métricas y lo convierte a un archivo XLSX.
    """
    if not os.path.exists(ruta_csv):
        print(f"Advertencia: El archivo CSV no fue encontrado en la ruta: {ruta_csv}")
        return False

    try:
        # 1. Leer el archivo CSV con pandas
        df = pd.read_csv(ruta_csv)

        # 2. Guardar el DataFrame como archivo Excel
        df.to_excel(ruta_excel, index=False)

        print(f"Métricas de '{ruta_csv}' guardadas exitosamente en: {ruta_excel}")
        return True

    except Exception as e:
        print(f"Error durante la conversión de '{ruta_csv}' a Excel: {e}")
        return False

In [ ]:
from google.colab import files

# Lista de los archivos CSV de registro que se generaron
archivos_csv_registros = [
    'registro_entrenamiento_modelo_1_normal.csv',
    'registro_entrenamiento_modelo_1_opt.csv',
    'registro_entrenamiento_modelo_1_cplana.csv',
    'registro_entrenamiento_modelo_2.csv',
    'registro_entrenamiento_modelo_3.csv',
    'registro_entrenamiento_modelo_4.csv',
    'log_model_TL.csv'
]

# Carpeta para guardar los archivos Excel
carpeta_excel = '/content/Metricas_Excel/'
os.makedirs(carpeta_excel, exist_ok=True)

print("--- Iniciando conversión y descarga de métricas ---")
for nombre_csv in archivos_csv_registros:
    nombre_excel = nombre_csv.replace('.csv', '.xlsx')
    ruta_excel_destino = os.path.join(carpeta_excel, nombre_excel)

    # Intentar convertir el archivo
    if convertir_csv_a_excel(nombre_csv, ruta_excel_destino):
        # Si la conversión fue exitosa, descargar el archivo
        print(f"Iniciando descarga de {ruta_excel_destino}...")
        files.download(ruta_excel_destino)

print("\n--- Proceso de conversión y descarga finalizado. ---")

--- Iniciando conversión y descarga de métricas ---
Advertencia: El archivo CSV no fue encontrado en la ruta: registro_entrenamiento_modelo_1_normal.csv
Advertencia: El archivo CSV no fue encontrado en la ruta: registro_entrenamiento_modelo_1_opt.csv
Advertencia: El archivo CSV no fue encontrado en la ruta: registro_entrenamiento_modelo_1_cplana.csv
Advertencia: El archivo CSV no fue encontrado en la ruta: registro_entrenamiento_modelo_2.csv
Advertencia: El archivo CSV no fue encontrado en la ruta: registro_entrenamiento_modelo_3.csv
Métricas de 'registro_entrenamiento_modelo_4.csv' guardadas exitosamente en: /content/Metricas_Excel/registro_entrenamiento_modelo_4.xlsx
Iniciando descarga de /content/Metricas_Excel/registro_entrenamiento_modelo_4.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Métricas de 'log_model_TL.csv' guardadas exitosamente en: /content/Metricas_Excel/log_model_TL.xlsx
Iniciando descarga de /content/Metricas_Excel/log_model_TL.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Proceso de conversión y descarga finalizado. ---


In [ ]:
import os

# Crear la carpeta para guardar los modelos si no existe
os.makedirs('/content/MisModelos', exist_ok=True)

# Guardar los modelos entrenados
print("Guardando modelos...")
Modelo_1_Normal_Refac.save('/content/MisModelos/Modelo_1_Normal.h5')
Modelo_1_Optimizado_Refac.save('/content/MisModelos/Modelo_1_Optimizado.h5')
Modelo_1_CapaPlana_Refac.save('/content/MisModelos/Modelo_1_CapaPlana.h5')
Modelo_2_Refac.save('/content/MisModelos/Modelo_2.h5')
Modelo_3_Refac.save('/content/MisModelos/Modelo_3.h5')
Modelo_4_Refac.save('/content/MisModelos/Modelo_4.h5')

print("Modelos guardados exitosamente en /content/MisModelos/")

Guardando modelos...
Modelos guardados exitosamente en /content/MisModelos/
